In [ ]:
!pip install albumentations==0.5.2
!pip install efficientnet_pytorch
!pip install pycocotools
!pip install cython
!pip install timm
!pip install effdet
!pip install gluoncv
!pip install mxnet
!pip install wandb
!pip install madgrad

In [ ]:
# !wget https://prod-aistages-public.s3.amazonaws.com/app/Competitions/000035/data/data.tar.gz

In [ ]:
# import os
# os.chdir('/content/input/data')
# !tar -xzf /content/data.tar.gz

In [ ]:
from pycocotools.coco import COCO
import gluoncv.utils as gcv
import torch
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain , DetBenchPredict
from effdet.efficientdet import HeadNet
import os
from datetime import datetime
import time
import random
import wandb
import cv2
import pandas as pd
import numpy as np
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from tqdm.auto import tqdm
import warnings
from torch.optim.lr_scheduler import _LRScheduler
import math
from scheduler import *
from madgrad import MADGRAD
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
batch_size = 6
learning_rate = 1e-3
max_lr = learning_rate
min_lr = 1e-6
num_epochs = 100
seed = 2021
seed_everything(seed)

In [ ]:
IMAGENET_DEFAULT_MEAN = [x * 255 for x in (0.485, 0.456, 0.406)]
IMAGENET_DEFAULT_STD = [x * 255 for x in (0.229, 0.224, 0.225)]

class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''
    def __init__(self, annotation, data_dir, transforms , type = 'train'):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms
        self.type = type

    def __getitem__(self, index: int):
        image_id = self.coco.getImgIds(imgIds=index)
        image, boxes, labels = self.load_image_and_boxes(index)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        
        # Multi Scale
        target['img_size'] = torch.tensor([(512, 512)])
        target['img_scale'] = torch.tensor([1.])
        
        if self.transforms:
            # Transform 적용 후 Box가 없다면 있을 때 까지 반복 (default = 10)
            for i in range(10):
                sample = {
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': target['labels']
                }
                sample = self.transforms(**sample)
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    target['labels'] = torch.tensor(sample['labels'])
                    break
        
        return image, target, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())
    
    def load_image_and_boxes(self, index):
        image_id = self.coco.getImgIds(imgIds=index)
        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = (image - IMAGENET_DEFAULT_MEAN) / IMAGENET_DEFAULT_STD

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        boxes = np.array([x['bbox'] for x in anns])
        
        labels = np.array([x['category_id'] for x in anns])
        #labels = torch.as_tensor(labels, dtype=torch.int64)

        # boxex (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        return image, boxes, labels

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

def get_train_transform():
    return A.Compose([
        A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels'])
    )

def get_valid_transform():
    return A.Compose([
        A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels'])
    )

train_annotation = '/content/input/data/train.json'
val_annotation = '/content/input/data/val.json'
data_dir = '/content/input/data'
train_dataset = CustomDataset(train_annotation, data_dir, get_train_transform())
val_dataset = CustomDataset(val_annotation, data_dir, get_valid_transform())

train_data_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn,num_workers=2)
val_data_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=False,collate_fn=collate_fn,num_workers=2)

loading annotations into memory...
Done (t=3.84s)
creating index...
index created!
loading annotations into memory...
Done (t=0.87s)
creating index...
index created!


In [ ]:
def get_model():
    # get weight
    # url : https://github.com/rwightman/efficientdet-pytorch
    config = get_efficientdet_config('tf_efficientdet_d4')
    config.image_size = (512, 512)
    config.norm_kwargs=dict(eps=.001, momentum=.01)
    net = EfficientDet(config, pretrained_backbone=True)
    checkpoint = torch.load('/content/drive/MyDrive/efficientdet_d4-5b370b7a.pth')
    net.load_state_dict(checkpoint)

    net.reset_head(num_classes=11)
    net.class_net = HeadNet(config, num_outputs=11)
    return DetBenchTrain(net, config)

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class EarlyStopping:
    def __init__(self, patience=50, verbose=False, delta=0, path=None):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        self.path = path

    def __call__(self, val_mAP, model):
        score = val_mAP
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        model.eval()
        torch.save(model.state_dict(), self.path)

In [ ]:
def get_learning_rate(optimizer):
    lr = []
    for param_group in optimizer.param_groups:
        lr += [param_group['lr']]
    return lr

In [ ]:
def train_fn(model, train_data_loader , val_data_loader, optimizer,scheduler, device):
    model.train()
    train_loss = AverageMeter()
    train_cls_loss = AverageMeter()
    train_box_loss = AverageMeter()

    save_path = '/content/drive/MyDrive/saved/checkpoint.pth'
    early_stop = EarlyStopping(path=save_path)
    for epoch in range(num_epochs):
        ################################# train ###############################
        train_loss.reset()
        train_cls_loss.reset()
        train_box_loss.reset()
        model.train()
        for images, targets, image_ids in tqdm(train_data_loader):
            # gpu 계산을 위해 image.to(device)
            images = torch.stack(images)
            images = images.to(device).float()
            current_batch_size = images.shape[0]

            targets2 = {}
            targets2['bbox'] = [target['boxes'].to(device).float() for target in targets] # variable number of instances, so the entire structure can be forced to tensor
            targets2['cls'] = [target['labels'].to(device).float() for target in targets]
            targets2['image_id'] = torch.tensor([target['image_id'] for target in targets]).to(device).float()
            targets2['img_scale'] = torch.tensor([target['img_scale'] for target in targets]).to(device).float()
            targets2['img_size'] = torch.tensor([(512, 512) for target in targets]).to(device).float()

            print(images)
            # calculate loss
            losses, cls_loss, box_loss = model(images, targets2).values()

            train_loss.update(losses.detach().item(), current_batch_size)
            train_cls_loss.update(cls_loss.detach().item(), current_batch_size)
            train_box_loss.update(box_loss.detach().item(), current_batch_size)

            # backward
            optimizer.zero_grad()
            losses.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 10.0)
            optimizer.step()
            if scheduler is not None:
                scheduler.step()
            wandb.log({'Learning rate': get_learning_rate(optimizer)[0]})       

        # Evaluation
        val_loss, val_mAP = vali_fn(val_data_loader,model,device)
        
        print(f"\nEpoch #{epoch+1} train loss: [{train_loss.avg}] train cls loss : [{train_cls_loss.avg}] train box loss : [{train_box_loss.avg}] validation mAP : [{val_mAP}] \n")
        wandb.log({"Train Loss":train_loss.avg, "Validation mAP@50":val_mAP})
        early_stop(val_mAP,model)
        if early_stop.early_stop:
            print('Stop Training.....')
            break

In [ ]:
def vali_fn(val_data_loader, model, device):
    model.eval()
    vali_loss = AverageMeter()
    vali_mAP = AverageMeter()
    # Custom
    metric = gcv.metrics.VOCMApMetric(iou_thresh=0.5)
    with torch.no_grad():
        for images, targets, image_ids in tqdm(val_data_loader):
            # gpu 계산을 위해 image.to(device)
            images = torch.stack(images)
            images = images.to(device).float()

            current_batch_size = images.shape[0]

            targets2 = {}
            targets2['bbox'] = [target['boxes'].to(device).float() for target in targets] # variable number of instances, so the entire structure can be forced to tensor
            targets2['cls'] = [target['labels'].to(device).float() for target in targets]
            targets2['image_id'] = torch.tensor([target['image_id'] for target in targets]).to(device).float()
            targets2['img_scale'] = torch.tensor([target['img_scale'] for target in targets]).to(device).float()
            targets2['img_size'] = torch.tensor([(512, 512) for target in targets]).to(device).float()

            outputs = model(images, targets2)

            loss = outputs['loss']
            det = outputs['detections']

            # Calc Metric
            for i in range(0, len(det)):
                pred_scores=det[i, :, 4].cpu().unsqueeze_(0).numpy()
                condition=(pred_scores > 0.05)[0]
                gt_boxes=targets2['bbox'][i].cpu().unsqueeze_(0).numpy()[...,[1,0,3,2]] #move to PASCAL VOC from yxyx format
                gt_labels=targets2['cls'][i].cpu().unsqueeze_(0).numpy()

                pred_bboxes=det[i, :, 0:4].cpu().unsqueeze_(0).numpy()[:, condition, :]
                pred_labels=det[i, :, 5].cpu().unsqueeze_(0).numpy()[:, condition]
                pred_scores=pred_scores[:, condition]
                metric.update(
                  pred_bboxes=pred_bboxes,
                  pred_labels=pred_labels,
                  pred_scores=pred_scores,
                  gt_bboxes=gt_boxes,
                  gt_labels=gt_labels)

            vali_mAP.update(metric.get()[1], current_batch_size)
            vali_loss.update(loss.detach().item(), current_batch_size)
    
    # validation loss
    return vali_loss.avg, vali_mAP.avg

In [ ]:
# Model
model = get_model()
model.load_state_dict(torch.load('/content/drive/MyDrive/saved/checkpoint.pth'))
model.to(device)
model.eval()
# setting
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=learning_rate, momentum = 0.9)
scheduler = None
# first_cycle_steps = len(train_data_loader)* num_epochs // 2
# scheduler = CosineAnnealingWarmupRestarts(
#     optimizer, 
#     first_cycle_steps=first_cycle_steps, 
#     cycle_mult=1.0, 
#     max_lr=max_lr, 
#     min_lr=min_lr, 
#     warmup_steps=int(first_cycle_steps * 0.25), 
#     gamma=0.5
# )


In [ ]:
wandb.login()
wandb.init(project='P-stage3-detection', name="EfficientDet")
train_fn(model, train_data_loader , val_data_loader, optimizer, scheduler , device)

# Inference

In [ ]:
class TestDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir, transforms=None):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.transforms = transforms

    def __getitem__(self, index: int):
        
        image_id = self.coco.getImgIds(imgIds=index)

        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        # transform
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']

        return image , image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [ ]:
def get_model():
    # get weight
    # url : https://github.com/rwightman/efficientdet-pytorch
    config = get_efficientdet_config('tf_efficientdet_d4')
    config.image_size = (512, 512)
    config.norm_kwargs=dict(eps=.001, momentum=.01)
    net = EfficientDet(config, pretrained_backbone=True)

    net.reset_head(num_classes=11)
    net.class_net = HeadNet(config, num_outputs=11)
    return DetBenchPredict(net)

In [ ]:
def predict(test_data_loader, model, device):
    model.eval()
    outputs = []
    with torch.no_grad():
        for images, image_ids in tqdm(test_data_loader):
            # gpu 계산을 위해 image.to(device)
            images = torch.stack(images)
            images = images.to(device).float()

            current_batch_size = images.shape[0]

            det = model(images)
              
            # Calc Metric
            # 좌표, cs , label
            for i in range(0, len(det)):
                pred_scores=det[i, :, 4].cpu().unsqueeze_(0).numpy()
                condition=(pred_scores > 0.00)[0]
                pred_bboxes=det[i, :, 0:4].cpu().unsqueeze_(0).numpy()[:, condition, :]
                pred_labels=det[i, :, 5].cpu().unsqueeze_(0).numpy()[:, condition]
                pred_scores=pred_scores[:, condition]
                outputs.append({'boxes': pred_bboxes.tolist(), 'scores': pred_scores.tolist(), 'labels': pred_labels.tolist()})

    return outputs

In [ ]:
def infernece(model , test_data_loader, device):
    model.eval()
    outputs = predict(test_data_loader, model, device)
    prediction_strings = []
    file_names = []
    coco = COCO(test_annotation)
    score_threshold = 0.00
    for i, output in enumerate(outputs):   
        prediction_string = ''
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        for box, score, label in zip(output['boxes'][0], output['scores'][0], output['labels'][0]): 
            if score > score_threshold and  all(0<= elem <=512 for elem in box) :
                  prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(
                      box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '

        prediction_strings.append(prediction_string)
        file_names.append(image_info['file_name'])
    submission = pd.DataFrame()
    submission['PredictionString'] = prediction_strings
    submission['image_id'] = file_names
    submission.to_csv(f'Efficientdet_submission.csv', index=None)
    print(submission.head())

In [ ]:
def get_test_transform():
    return A.Compose([
        A.Resize(512, 512 , p =1.0),
        ToTensorV2(p=1.0)
    ])

test_annotation = '/content/input/data/test.json'
test_data_dir = '/content/input/data'
test_dataset = TestDataset(test_annotation, test_data_dir, get_test_transform())

test_data_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)

#Model
model = get_model()
model.load_state_dict(torch.load('/content/drive/MyDrive/saved/checkpoint.pth'))
model.to(device)

# inference
infernece(model, test_data_loader, device)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!



loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString              image_id
0  8 0.9980921149253845 173.08245849609375 236.41...  batch_01_vt/0021.jpg
1  8 0.557726263999939 285.8876647949219 250.9930...  batch_01_vt/0028.jpg
2  1 0.9940304160118103 75.79487609863281 169.357...  batch_01_vt/0031.jpg
3  8 0.070650115609169 296.2857971191406 266.9136...  batch_01_vt/0032.jpg
4  7 0.9985429048538208 0.5882797241210938 86.769...  batch_01_vt/0070.jpg


In [ ]:
df = pd.read_csv('/content/input/data/Efficientdet_submission.csv')
df['PredictionString'][0]

'8 0.9980921149253845 173.08245849609375 236.4196014404297 467.82977294921875 493.4827880859375 7 0.09756028652191162 133.5152587890625 49.27411651611328 257.794189453125 171.14093017578125 3 0.09317931532859802 134.34291076660156 47.38205337524414 256.5142822265625 166.55084228515625 7 0.05943967401981354 139.29913330078125 47.24940490722656 341.6982421875 231.56031799316406 2 0.05539894104003906 133.24815368652344 48.06289291381836 257.19000244140625 169.51516723632812 3 0.053076907992362976 135.30203247070312 51.81950378417969 334.4064025878906 233.2051544189453 7 0.048682838678359985 174.8128662109375 231.37738037109375 465.49237060546875 488.66650390625 6 0.04674661532044411 424.21710205078125 198.18186950683594 511.4564208984375 231.6643829345703 1 0.04479493573307991 133.5152587890625 49.27411651611328 257.794189453125 171.14093017578125 2 0.039826467633247375 178.88735961914062 236.97039794921875 474.1406555175781 490.8555908203125 6 0.03944314271211624 178.88735961914062 236.9